In [1]:
# Michel Bierlaire
# Sun Oct 21 23:05:21 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable

df = pd.read_csv("optimaDataset.dat",'\t')
database = db.Database("optimaDataset",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)

exclude =  ((Choice   ==  2 ) + (Choice == -1) + (  CostCarCHF   <=  0  )) >0
database.remove(exclude)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed

ASC_CAR	 = Beta('ASC_CAR',0,None,None,1)
ASC_PT_SHORT	 = Beta('ASC_PT_SHORT',0,None,None,0)
ASC_PT_LONG	 = Beta('ASC_PT_LONG',0,None,None,0)
BETA_BURDEN	 = Beta('BETA_BURDEN',0,None,None,0)
BETA_COST_CAR	 = Beta('BETA_COST_CAR',0,None,None,0)
BETA_COST_PT	 = Beta('BETA_COST_PT',0,None,None,0)
BETA_TIME	 = Beta('BETA_TIME',0,None,None,0)


ASC_CAR_FRENCH	 = Beta('ASC_CAR_FRENCH',0,None,None,1)
ASC_PT_SHORT_FRENCH	 = Beta('ASC_PT_SHORT_FRENCH',0,None,None,0)
ASC_PT_LONG_FRENCH	 = Beta('ASC_PT_LONG_FRENCH',0,None,None,0)
BETA_BURDEN_FRENCH	 = Beta('BETA_BURDEN_FRENCH',0,None,None,0)
BETA_COST_CAR_FRENCH	 = Beta('BETA_COST_CAR_FRENCH',0,None,None,0)
BETA_COST_PT_FRENCH	 = Beta('BETA_COST_PT_FRENCH',0,None,None,0)
BETA_TIME_FRENCH	 = Beta('BETA_TIME_FRENCH',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

DistanceLessThan23  = DefineVariable('DistanceLessThan23', distance_km   <  23 ,database)
Burden  = DefineVariable('Burden',(  WalkingTimePT   +  WaitingTimePT   ) / (  NbTransf   +  1  ),database)
age_yo  = DefineVariable('age_yo',(  age   >=  65  ) + (  age   <=  30  ),database)
MissingAge  = DefineVariable('MissingAge', age   <  1 ,database)
French  = DefineVariable('French', LangCode   ==  1 ,database)

#Utilities
CAR_OTHER = ASC_CAR + BETA_COST_CAR * CostCarCHF + BETA_TIME * TimeCar
PT_OTHER = ASC_PT_SHORT * DistanceLessThan23 +  ASC_PT_LONG * (1-DistanceLessThan23) + BETA_COST_PT * MarginalCostPT + BETA_TIME * TimePT  + BETA_BURDEN * Burden

CAR_FRENCH = ASC_CAR_FRENCH + BETA_COST_CAR_FRENCH * CostCarCHF + BETA_TIME_FRENCH * TimeCar
PT_FRENCH = ASC_PT_SHORT_FRENCH * DistanceLessThan23 +  ASC_PT_LONG_FRENCH * (1-DistanceLessThan23) + BETA_COST_PT_FRENCH * MarginalCostPT + BETA_TIME_FRENCH * TimePT  + BETA_BURDEN_FRENCH * Burden

CAR = CAR_FRENCH * French + CAR_OTHER * (1-French)
PT = PT_FRENCH * French + PT_OTHER * (1-French)

V = {1: CAR,0: PT}
av = {1: 1,0: 1}

# Logit model, with availability conditions
logprob = models.loglogit(V,av,Choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "binary_socio_econ_optima"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


                        Value  Std err  t-test  p-value  Rob. Std err  \
ASC_PT_LONG             0.149     0.18    0.83    0.407         0.198   
ASC_PT_LONG_FRENCH       0.13    0.507   0.257    0.798         0.502   
ASC_PT_SHORT          -0.0951    0.134  -0.708    0.479         0.143   
ASC_PT_SHORT_FRENCH     -0.82    0.365   -2.25   0.0247         0.344   
BETA_BURDEN          -0.00815  0.00494   -1.65   0.0987       0.00631   
BETA_BURDEN_FRENCH    -0.0189   0.0133   -1.42    0.156         0.013   
BETA_COST_CAR          -0.125   0.0151   -8.27 2.22e-16         0.027   
BETA_COST_CAR_FRENCH   -0.209   0.0414   -5.05 4.34e-07        0.0422   
BETA_COST_PT          -0.0615  0.00868   -7.09 1.34e-12        0.0169   
BETA_COST_PT_FRENCH   -0.0862   0.0203   -4.24 2.27e-05        0.0237   
BETA_TIME            -0.00984  0.00188   -5.23 1.71e-07       0.00356   
BETA_TIME_FRENCH      -0.0223  0.00521   -4.28 1.85e-05       0.00481   

                      Rob. t-test  Rob. p-value  


/usr/local/lib/python3.7/site-packages/scipy/linalg/basic.py:1321: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  x, resids, rank, s = lstsq(a, b, cond=cond, check_finite=False)
